In [15]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib #jbolib模块
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [22]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [3]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\EBV")

In [6]:
Raw_data1= pd.read_csv('./AID_1950_datatable_all.csv',index_col=0)
Raw_data2= pd.read_csv('./AID_2234_datatable_all.csv',index_col=0)

In [8]:
df = pd.concat([Raw_data1,Raw_data2])
df.dropna(subset=["PUBCHEM_CID"], inplace=True)  
df['Activity']=np.where(df['PUBCHEM_ACTIVITY_OUTCOME']=='Active', 1, 0)
df.sort_values(by="Activity" , inplace=True, ascending=False) 
df.drop_duplicates(subset='PUBCHEM_CID', keep='first', inplace=True)
df['cid']=df['PUBCHEM_CID']
df.index=df['cid']
df=df.drop(['PUBCHEM_CID'], axis=1)
df=df.drop(['PUBCHEM_SID'], axis=1)
df=df.drop(['cid'], axis=1)
df

,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,Inhibition,Activity
cid,,,,,,,
5702697.0,CCN\1C2=CC=CC=C2S/C1=C\C=C\C=C\C3=[N+](C4=CC=C...,Active,100.0,NaN,NaN,1126.69,1
4294204.0,CC1=C(C=C(C=C1)NC(=S)NC(=O)C2=CC=C(C=C2)F)S(=O...,Active,2.0,NaN,NaN,27.78,1
3185712.0,CC1=CC=CC=C1N(C(C2=CC=C(C=C2)N(C)C)C(=O)NCC3CC...,Active,2.0,NaN,NaN,27.61,1
135459016.0,CCOC1=C(C=CC(=C1)/C=N/NC(=O)C(=O)N/N=C/C2=CC(=...,Active,2.0,NaN,NaN,27.69,1
3532333.0,C1=CC=C2C(=C1)C(=O)N(C(=S)N2)CCC(=O)NC3=CC(=CC...,Active,2.0,NaN,NaN,27.71,1
...,...,...,...,...,...,...,...
3295.0,CCOC1=CC2=C(C=C1)N=C(S2)S(=O)(=O)N,Inactive,0.0,NaN,NaN,9.84,0
2343528.0,C1=CC=C(C=C1)NC2=NC(=CS2)COC(=O)COC3=CC=C(C=C3)Cl,Inactive,0.0,NaN,NaN,9.84,0
664262.0,C1C(ON=C1C2=CC=CC=C2)CN3C(=O)C4=CC=CC=C4S3(=O)=O,Inactive,0.0,NaN,NaN,9.84,0


In [9]:
df['IsomericSMILES']=df['PUBCHEM_EXT_DATASOURCE_SMILES']
Original_data=pd.DataFrame(df['IsomericSMILES'])
Original_data['Activity']=df['Activity']
Chemdraw_smiles=Original_data['IsomericSMILES'].tolist()

Original_data.head()

,IsomericSMILES,Activity
cid,,
5702697.0,CCN\1C2=CC=CC=C2S/C1=C\C=C\C=C\C3=[N+](C4=CC=C...,1
4294204.0,CC1=C(C=C(C=C1)NC(=S)NC(=O)C2=CC=C(C=C2)F)S(=O...,1
3185712.0,CC1=CC=CC=C1N(C(C2=CC=C(C=C2)N(C)C)C(=O)NCC3CC...,1
135459016.0,CCOC1=C(C=CC(=C1)/C=N/NC(=O)C(=O)N/N=C/C2=CC(=...,1
3532333.0,C1=CC=C2C(=C1)C(=O)N(C(=S)N2)CCC(=O)NC3=CC(=CC...,1


In [10]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Original_data['Canonical_smiles']=Canonical_smis
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Original_data

,IsomericSMILES,Activity,Canonical_smiles
cid,,,
5702697.0,CCN\1C2=CC=CC=C2S/C1=C\C=C\C=C\C3=[N+](C4=CC=C...,1,CCN1/C(=C/C=C/C=C/c2sc3ccccc3[n+]2CC)Sc2ccccc2...
4294204.0,CC1=C(C=C(C=C1)NC(=S)NC(=O)C2=CC=C(C=C2)F)S(=O...,1,Cc1ccc(NC(=S)NC(=O)c2ccc(F)cc2)cc1S(=O)(=O)Nc1...
3185712.0,CC1=CC=CC=C1N(C(C2=CC=C(C=C2)N(C)C)C(=O)NCC3CC...,1,Cc1ccccc1N(C(=O)c1snc(C(N)=O)c1N)C(C(=O)NCC1CC...
135459016.0,CCOC1=C(C=CC(=C1)/C=N/NC(=O)C(=O)N/N=C/C2=CC(=...,1,CCOc1cc(/C=N/NC(=O)C(=O)N/N=C/c2ccc(O)c(OCC)c2...
3532333.0,C1=CC=C2C(=C1)C(=O)N(C(=S)N2)CCC(=O)NC3=CC(=CC...,1,O=C(CCn1c(=S)[nH]c2ccccc2c1=O)Nc1cccc(Cl)c1
...,...,...,...
3295.0,CCOC1=CC2=C(C=C1)N=C(S2)S(=O)(=O)N,0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
2343528.0,C1=CC=C(C=C1)NC2=NC(=CS2)COC(=O)COC3=CC=C(C=C3)Cl,0,O=C(COc1ccc(Cl)cc1)OCc1csc(Nc2ccccc2)n1
664262.0,C1C(ON=C1C2=CC=CC=C2)CN3C(=O)C4=CC=CC=C4S3(=O)=O,0,O=C1c2ccccc2S(=O)(=O)N1CC1CC(c2ccccc2)=NO1


In [4]:
data_psm= pd.read_csv('./data_psm.csv',index_col=0)
data_psm

,IsomericSMILES,Activity,Canonical_smiles
OPTUM_LAB_ID,,,
3185712.0,CC1=CC=CC=C1N(C(C2=CC=C(C=C2)N(C)C)C(=O)NCC3CC...,1,Cc1ccccc1N(C(=O)c1snc(C(N)=O)c1N)C(C(=O)NCC1CC...
3532333.0,C1=CC=C2C(=C1)C(=O)N(C(=S)N2)CCC(=O)NC3=CC(=CC...,1,O=C(CCn1c(=S)[nH]c2ccccc2c1=O)Nc1cccc(Cl)c1
16830460.0,CN1CCN(CC1)C2=NN=C(C=C2)C3=CC(=CC=C3)NC(=O)COC...,1,CN1CCN(c2ccc(-c3cccc(NC(=O)COc4ccc(Cl)cc4Cl)c3...
3921421.0,CC1=CC(=NC(=N1)OC(C(=O)O)C(C2=CC=CC=C2)(C3=CC=...,1,Cc1cc(C)nc(OC(C(=O)O)C(OCCc2ccccc2)(c2ccccc2)c...
1314610.0,C1OC2=CC=CC(=C2O1)CNC3=C(N=C(O3)C4=CC=C(C=C4)C...,1,N#Cc1nc(-c2ccc(Cl)cc2)oc1NCc1cccc2c1OCO2
...,...,...,...
664630.0,CCC(=O)N(CC1=CC2=CC=CC=C2NC1=O)C(C)C,0,CCC(=O)N(Cc1cc2ccccc2[nH]c1=O)C(C)C
3228536.0,CN1C2=C(C=C1C(=O)OC(C3=CC=NC=C3)C(=O)NCC4=CC=C...,0,Cn1c(C(=O)OC(C(=O)NCc2ccco2)c2ccncc2)cc2sccc21
666815.0,CN1C2=C(C(=O)N(C1=O)C)N(C(=N2)CN3CCOCC3)CC(=O)...,0,Cn1c(=O)c2c(nc(CN3CCOCC3)n2CC(=O)c2ccccc2)n(C)...


In [5]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [10]:
#Calculate the descriotors
aDesc.set_input_SMILES(data_psm['Canonical_smiles'].tolist())
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [11]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=data_psm.index

X_original_data.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
3185712.0,536.72,7.667429,44.0231,70.4832,46.3981,79.1450,0.628901,1.006903,0.662830,1.130643,...,0.0,3.0,0.0,14.0,31.5,0.828947,9.5,0.250000,40.917043,12.132410
3532333.0,359.86,9.470000,26.6692,38.6617,28.0001,42.2689,0.701821,1.017413,0.736845,1.112339,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
16830460.0,472.41,8.589273,36.4596,55.6470,38.2673,61.9503,0.662902,1.011764,0.695769,1.126369,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
3921421.0,468.59,7.437937,40.7504,62.9996,42.7276,70.2318,0.646832,0.999994,0.678216,1.114790,...,0.0,0.0,6.0,18.0,31.0,0.885714,4.0,0.114286,33.763886,4.472136
1314610.0,353.78,9.561622,26.6701,38.0290,27.0455,41.1431,0.720814,1.027811,0.730959,1.111976,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000


In [12]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
X_NAomit_data.head()
X_NAomit_data.to_csv("./X_NAomit_data.csv",sep=',',header=1,index=1)

In [16]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Activity"]=data_psm['Activity']
Notgelable=Ranksums_data.loc[Ranksums_data["Activity"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Activity"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Activity',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results

,Statistics,Pvalues
Ui,-12.858634,7.692395e-38
Eta_betaP,-12.596586,2.204820e-36
SM6_B(p),-12.156568,5.293652e-34
piID,-11.994484,3.797782e-33
SM04_EA(bo),-11.821243,3.031655e-32
...,...,...
B09[S-B],0.000000,1.000000e+00
B09[S-I],0.000000,1.000000e+00
CATS2D_05_PP,0.000000,1.000000e+00
B09[P-I],0.000000,1.000000e+00


In [17]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(1464, 2)


,Statistics,Pvalues
Ui,-12.858634,7.692395e-38
Eta_betaP,-12.596586,2.204820e-36
SM6_B(p),-12.156568,5.293652e-34
piID,-11.994484,3.797782e-33
SM04_EA(bo),-11.821243,3.031655e-32


In [18]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
X_ranksums_data.head()

,Ui,Eta_betaP,SM6_B(p),piID,SM04_EA(bo),SM5_B(p),SM6_B(m),SM6_B(v),Uc,nBM,...,CATS2D_04_DD,Eig15_EA(dm),PHI,SM14_EA(ri),JGI10,MATS8i,SpMin6_Bh(v),SM15_AEA(ed),F09[N-N],NssO
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
3185712.0,3.643856,22.0,9.418112,9.471186,7.341079,8.182111,9.769582,9.429444,4.392317,20.0,...,1.0,0.0,9.025193,16.730952,0.007342,-0.052743,1.688588,29.925224,1.0,1.0
3532333.0,3.321928,17.0,9.032067,9.186038,6.902240,7.805473,9.502513,9.025803,4.000000,15.0,...,0.0,0.0,5.138558,15.850933,0.005096,0.014787,1.177822,28.845948,0.0,0.0
16830460.0,3.459432,21.0,9.200365,11.122681,7.179879,7.978777,9.656455,9.232732,4.321928,19.0,...,0.0,0.0,7.477700,15.255271,0.007392,0.099787,1.475902,28.391020,0.0,1.0
3921421.0,3.807355,26.0,9.349937,9.551339,7.309212,8.125985,9.501375,9.382866,4.700440,25.0,...,0.0,0.0,7.790438,17.865523,0.006472,0.133047,1.580635,31.680336,0.0,2.0
1314610.0,3.523562,20.5,9.103018,10.543188,7.154908,7.856176,9.463106,9.145338,4.247928,18.0,...,0.0,0.0,4.326573,15.838385,0.004306,0.088138,1.270929,28.798699,0.0,2.0


In [20]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=data_psm.index
X_scaled_data.to_csv("./X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Scaler_transformer.pkl')
X_scaled_data.head()

,Ui,Eta_betaP,SM6_B(p),piID,SM04_EA(bo),SM5_B(p),SM6_B(m),SM6_B(v),Uc,nBM,...,CATS2D_04_DD,Eig15_EA(dm),PHI,SM14_EA(ri),JGI10,MATS8i,SpMin6_Bh(v),SM15_AEA(ed),F09[N-N],NssO
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
3185712.0,0.660860,0.4750,0.666072,0.351500,0.631196,0.650365,0.227632,0.637375,0.712780,0.418605,...,0.25,0.84483,0.176391,0.498928,0.356238,0.344442,0.780514,0.522476,0.125,0.166667
3532333.0,0.557528,0.3500,0.478613,0.323871,0.430159,0.463049,0.181383,0.437487,0.613172,0.302326,...,0.00,0.84483,0.080920,0.365366,0.247234,0.418779,0.389447,0.378264,0.000,0.000000
16830460.0,0.601664,0.4500,0.560336,0.511522,0.557349,0.549239,0.208041,0.539961,0.694908,0.395349,...,0.00,0.84483,0.138379,0.274961,0.358655,0.512348,0.617672,0.317477,0.000,0.166667
3921421.0,0.713339,0.5750,0.632967,0.359267,0.616598,0.622451,0.181186,0.614309,0.791011,0.534884,...,0.00,0.84483,0.146061,0.671125,0.314003,0.548961,0.697860,0.756994,0.000,0.333333
1314610.0,0.622248,0.4375,0.513066,0.455372,0.545910,0.488266,0.174559,0.496682,0.676120,0.372093,...,0.00,0.84483,0.060975,0.363461,0.208924,0.499524,0.460734,0.371951,0.000,0.333333


In [24]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=data_psm.index
X_cor_data.to_csv("./ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

1146 Moleculars × 218 Descriptors


,Ui,QED,TPSA(Tot),NaasC,ESOL,SM12_AEA(ri),Mv,SpMax2_Bh(m),SpMax2_Bh(e),SM1_Dz(Z),...,nR#C-,F02[C-N],GATS4e,C-032,Eig14_EA(dm),MATS5m,CATS2D_04_DD,Eig15_EA(dm),JGI10,F09[N-N]
OPTUM_LAB_ID,,,,,,,,,,,,,,,,,,,,,
3185712.0,0.660860,0.328626,0.408466,0.500000,0.556763,0.623705,0.309924,0.153048,0.654832,0.549136,...,0.0,0.384615,0.405201,0.000000,0.749278,0.594239,0.25,0.84483,0.356238,0.125
3532333.0,0.557528,0.816418,0.194061,0.285714,0.537555,0.501609,0.605445,0.166745,0.670674,0.485696,...,0.0,0.179487,0.600165,0.000000,0.749278,0.288806,0.00,0.84483,0.247234,0.000
16830460.0,0.601664,0.630167,0.110838,0.500000,0.391918,0.610567,0.447717,0.155370,0.659322,0.554790,...,0.0,0.333333,0.369052,0.000000,0.749278,0.585279,0.00,0.84483,0.358655,0.000
3921421.0,0.713339,0.434558,0.142937,0.428571,0.377472,0.575189,0.382590,0.119268,0.665461,0.232338,...,0.0,0.102564,0.581157,0.333333,0.749278,0.456585,0.00,0.84483,0.314003,0.000
1314610.0,0.622248,0.783865,0.139331,0.571429,0.508414,0.685740,0.682416,0.146742,0.722104,0.391917,...,0.0,0.153846,0.706366,0.000000,0.749278,0.431378,0.00,0.84483,0.208924,0.000
